In [1]:
import os
import csv
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import ARIMA
import matplotlib.pyplot as plt
from darts.metrics import mape, mse
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import ParameterGrid
from darts.dataprocessing.transformers import Scaler
from darts.models import TiDEModel, BlockRNNModel, NBEATSModel, XGBModel, RegressionModel, LinearRegressionModel, RandomForest

/Users/kim/Documents/Github/Insight-Wave/.venv/lib/python3.10/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
/Users/kim/Documents/Github/Insight-Wave/.venv/lib/python3.10/site-packages/statsforecast/utils.py:237: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  "ds": pd.date_range(start="1949-01-01", periods=len(AirPassengers), freq="M"),


In [2]:
def calculate_directional_accuracy(actual, forecast):  
    acc = 0
    for i in range(len(forecast)):
        actual_change = actual[i + 1] - actual[i]
        predicted_change = forecast[i] - actual[i]
        if (actual_change > 0 and predicted_change > 0) or (actual_change < 0 and predicted_change < 0) or (actual_change == 0 and predicted_change == 0):
            acc += 1

    da = round(acc / len(forecast) * 100, 2)
    return da

def calculate_directional_accuracy_with_thresholds(actual, forecast):
    thresholds = [0, 5, 10] 
    combinations = [(up, -down) for up in thresholds for down in thresholds if not (up > 0 and down > 0)]
    results = []
    for up, down in combinations:
        acc = 0
        for i in range(len(forecast)):
            actual_change = ((actual[i + 1] - actual[i]) / actual[i]) * 100
            predicted_change = ((forecast[i] - actual[i]) / actual[i]) * 100
            if (actual_change > up and predicted_change > up) or \
                (actual_change < down and predicted_change < down) or\
                (actual_change == predicted_change):
                acc += 1

        da = round(acc / len(forecast) * 100, 2)
        results.append((up, down, da))
    
    return results

def calculate_mape(actual, forecast):
    actual, forecast = np.array(actual), np.array(forecast)
    non_zero_actual = np.where(actual == 0, np.nan, actual)
    mape = np.mean(np.abs((actual - forecast) / non_zero_actual)) * 100
    return np.nanmean(mape)

def calculate_rmse(actual, forecast):
    actual, forecast = np.array(actual), np.array(forecast)
    if len(actual) != len(forecast):
        raise ValueError("The length of actual and forecast arrays must match.")
    mse = np.mean((forecast - actual) ** 2)
    return np.sqrt(mse)

def cal_err_and_acc(predicted_ts, val_ts, condition=True):
    val = val_ts.pd_dataframe()
    predicted = predicted_ts.pd_dataframe()

    val.reset_index(drop=True, inplace=True)
    predicted.reset_index(drop=True, inplace=True)

    actual = val['Close']
    forecast = predicted['Closing_Price']
    
    mape_error = calculate_mape(actual.iloc[1:], forecast)
    rmse_error = calculate_rmse(actual.iloc[1:], forecast)
    dir_acc = calculate_directional_accuracy_with_thresholds(actual, forecast)
    
    if condition:
        return mape_error, rmse_error, dir_acc
    else:
        print(f"MAPE = {mape_error:.2f} %")
        print(f"RMSE = {rmse_error:.2f} %\n")
        print(f"Directional Accuracy = {dir_acc:.2f} %")

In [3]:
stock = "CBG"
arima_predict = pd.read_csv(f"{os.getcwd()}/../backtest/{stock}/ARIMA.csv")
arima_predict_ts = TimeSeries.from_dataframe(arima_predict[['Closing_Price']], time_col=None)
ins_predict = pd.read_csv(f"/{os.getcwd()}/../backtest/{stock}/InsightWave.csv")
ins_predict_ts = TimeSeries.from_dataframe(ins_predict[['Closing_Price']], time_col=None)
val = pd.read_csv(f"{os.getcwd()}/../data/Fundamental+Technical Data/STOCK_DATA_WEEKLY/{stock}.csv")
val = val[['Close']].iloc[-35-1:]
val_ts = TimeSeries.from_dataframe(val, time_col=None)
arima_avg_mape, arima_avg_rmse, arima_avg_dir = cal_err_and_acc(arima_predict_ts, val_ts, True)
ins_avg_mape, ins_avg_rmse, ins_avg_dir = cal_err_and_acc(ins_predict_ts, val_ts, True)
print(f"ARIMA : MAPE {round(arima_avg_mape,2)}, RSME {round(arima_avg_rmse,2)}")
print(f"InsightWave : MAPE {round(ins_avg_mape,2)}, RSME {round(ins_avg_rmse,2)}")

FileNotFoundError: [Errno 2] No such file or directory: '/Users/kim/Documents/Github/Insight-Wave/StockPrediction/src/../backtest/CBG/ARIMA.csv'

In [52]:
def get_sim_fri_close(stock, mode, percentage):
    # Buy in Closing Price in Friday
    val = pd.read_csv(f"{os.getcwd()}/../data/Fundamental+Technical Data/STOCK_DATA_WEEKLY/{stock}.csv")
    val = val[['Date','Open','Close']].iloc[-35-1:].reset_index(drop=True)
    predict = pd.read_csv(f"{os.getcwd()}/../backtest/{stock}/{mode}.csv")
    predict = predict[['Date', 'Closing_Price']]

    # Backtest parameters
    starting_money = 1000000
    stock_count = 0
    money = starting_money
    buying_price = val.loc[0]['Close']

    for i in range(len(predict)):
        act = val.loc[i]['Close']
        pred = predict.loc[i]['Closing_Price']
        
        diff = ((pred - act) / act) * 100
        cut_lost = ((act - buying_price) / buying_price) * 100

        if diff > percentage:
            # Assuming you buy as much stock as possible with the money you have
            bought_stocks = money // act
            bought_stocks = int(bought_stocks / 100) * 100
            if money >= bought_stocks * act:
                money -= bought_stocks * act
                stock_count += bought_stocks
                buying_price = act
            #     print(f"Buy {bought_stocks} stocks at {act} on {val.loc[i]['Date']}, Money left: {money}")
            # else:
            #     print(f"Hold {stock_count} stocks at {act} on {val.loc[i]['Date']}, Money left: {money}")
        elif diff < -percentage and stock_count > 0:
            # Sell all stocks
            money += stock_count * act
            # print(f"Sell {stock_count} stocks at {act} on {val.loc[i]['Date']}, Money left: {money}")
            stock_count = 0
        elif cut_lost < -percentage and stock_count > 0:
            # Sell all stocks
            money += stock_count * act
            # print(f"Stop loss {stock_count} stocks at {act} on {val.loc[i]['Date']}, Money left: {money}")
            stock_count = 0
        # else:
        #     print(f"Hold {stock_count} stocks at {act} on {val.loc[i]['Date']}, Money left: {money}")

    # Sell any remaining stocks at the last available price
    final_money = money + stock_count * val.iloc[-1]['Close']
    profit_loss = final_money - starting_money

    return profit_loss

def get_sim_mon_open(stock, mode, percentage):
    # Buy in Closing Price in Friday
    val = pd.read_csv(f"{os.getcwd()}/../data/Fundamental+Technical Data/STOCK_DATA_WEEKLY/{stock}.csv")
    val = val[['Date','Open','Close']].iloc[-35-1:].reset_index(drop=True)
    predict = pd.read_csv(f"{os.getcwd()}/../backtest/{stock}/{mode}.csv")
    predict = predict[['Date', 'Closing_Price']]

    open_data = pd.read_csv(f"{os.getcwd()}/../data/Fundamental+Technical Data/STOCK_DATA/{stock}.csv")

    # Backtest parameters
    starting_money = 1000000
    stock_count = 0
    money = starting_money
    buying_price = val.loc[0]['Close']

    for i in range(len(predict)):
        specific_date_from_val = val.loc[i]['Date']
        today_row = open_data.index[open_data['Date'] == specific_date_from_val].to_list()[0]
        next_open_value = open_data.loc[today_row+1]['Open']
        next_date = open_data.loc[today_row+1]['Date']

        act = val.loc[i]['Close']
        pred = predict.loc[i]['Closing_Price']
        
        diff = ((pred - act) / act) * 100
        cut_lost = ((act - buying_price) / buying_price) * 100

        if diff > percentage:
            # Assuming you buy as much stock as possible with the money you have
            bought_stocks = money // next_open_value
            bought_stocks = int(bought_stocks / 100) * 100
            if money >= bought_stocks * next_open_value:
                money -= bought_stocks * next_open_value
                stock_count += bought_stocks
                buying_price = next_open_value
            #     print(f"Buy {bought_stocks} stocks at {next_open_value} on {next_date}, Money left: {money}")
            # else:
            #     print(f"Hold {stock_count} stocks at {next_open_value} on {next_date}, Money left: {money}")
        elif diff < -percentage and stock_count > 0:
            # Sell all stocks
            money += stock_count * next_open_value
            # print(f"Sell {stock_count} stocks at {next_open_value} on {next_date}, Money left: {money}")
            stock_count = 0
        elif cut_lost < -percentage and stock_count > 0:
            # Sell all stocks
            money += stock_count * next_open_value
            # print(f"Stop loss {stock_count} stocks at {act} on {val.loc[i]['Date']}, Money left: {money}")
            stock_count = 0
        # else:
        #     print(f"Hold {stock_count} stocks at {next_open_value} on {next_date}, Money left: {money}")

    # Sell any remaining stocks at the last available price
    final_money = money + stock_count * val.iloc[-1]['Close']
    profit_loss = final_money - starting_money

    return profit_loss

def DCA(stock):
    # Buy in Closing Price in Friday
    val = pd.read_csv(f"{os.getcwd()}/../data/Fundamental+Technical Data/STOCK_DATA_WEEKLY/{stock}.csv")
    val = val[['Date','Open','Close']].iloc[-35:].reset_index(drop=True)

    # DCA

    # Backtest parameters
    starting_money = 1000000
    stock_count = 0
    money = starting_money
    dca_interval = 4  # Buy every 4 data points
    allocated_money_per_buy = starting_money // ((35 // dca_interval)+1)

    # print(f"Initial Money: {starting_money}, Allocating {allocated_money_per_buy} per purchase")

    # Perform DCA strategy
    for i in range(len(val)):
        if i % dca_interval == 0 and money > 0:
            # Buy stocks every 4 weeks
            act = val.loc[i]['Close']
            bought_stocks = allocated_money_per_buy // act
            money -= bought_stocks * act
            stock_count += bought_stocks
            # print(f"Buy {bought_stocks} stocks at {act} on {val.loc[i]['Date']}, Money left: {money}")

    # Assuming selling all remaining stocks at the last available price
    final_money = money + stock_count * val.iloc[-1]['Close']
    profit_loss = final_money - starting_money
    return profit_loss

In [57]:
STOCK_LIST = ["ADVANC", "BANPU", "BH", "BTS", "CBG", "CPALL", "CPF", "INTUCH", "IVL", "KBANK", "LH", "PTT", "PTTEP", "PTTGC", "SCB", "SCC", "TISCO", "TU", "WHA"]
sum = 0
for stock in STOCK_LIST:
    percentage = 10
    # mon_arima = get_sim_mon_open(stock, "ARIMA", percentage)
    # mon_ins = get_sim_mon_open(stock, "InsightWave", percentage)
    # arima = get_sim_fri_close(stock, "ARIMA", percentage)
    ins = get_sim_fri_close(stock, "InsightWave", percentage)
    sum += ins
    # dca = DCA(stock)

    # if max(arima, ins, dca) == arima:
    #     best = "arima"
    # elif max(arima, ins, dca) == ins:
    #     best = "ins"
    # else:
    #     best = "dca"

    # print(f"{stock}, {mon_ins}, {ins}")

    # print(arima)
    # print("---")
    # print(ins)
    # print("---")
    # print(dca)
print(sum)

-127875.0


In [51]:
dt = get_sim_fri_close("BH", "InsightWave", 7)
dt

Hold 0 stocks at 213.0 on 2022-12-28, Money left: 1000000
Hold 0 stocks at 224.0 on 2023-01-09, Money left: 1000000
Hold 0 stocks at 208.0 on 2023-01-18, Money left: 1000000
Hold 0 stocks at 212.0 on 2023-01-27, Money left: 1000000
Hold 0 stocks at 214.0 on 2023-02-07, Money left: 1000000
Hold 0 stocks at 215.0 on 2023-02-16, Money left: 1000000
Hold 0 stocks at 214.0 on 2023-02-27, Money left: 1000000
Hold 0 stocks at 215.0 on 2023-03-09, Money left: 1000000
Hold 0 stocks at 211.0 on 2023-03-20, Money left: 1000000
Hold 0 stocks at 223.0 on 2023-03-29, Money left: 1000000
Hold 0 stocks at 236.0 on 2023-04-10, Money left: 1000000
Hold 0 stocks at 240.0 on 2023-04-21, Money left: 1000000
Hold 0 stocks at 243.0 on 2023-05-03, Money left: 1000000
Hold 0 stocks at 244.0 on 2023-05-16, Money left: 1000000
Buy 4300 stocks at 232.0 on 2023-05-25, Money left: 2400.0
Hold 4300 stocks at 234.0 on 2023-06-06, Money left: 2400.0
Hold 4300 stocks at 235.0 on 2023-06-15, Money left: 2400.0
Hold 4300

-60500.0